In [51]:
#import nltk
#nltk.download()#下载Model里的第一个包Averaged Perceptron Tagger

In [52]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import string
import copy
import json
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
data = pd.read_csv('News_Final.csv')[0:1000] #导入csv文件
samples = dict(data['Title'])#提取Headline列进行特征提取
samples

{0: 'Obama Lays Wreath at Arlington National Cemetery',
 1: 'A Look at the Health of the Chinese Economy',
 2: 'Nouriel Roubini: Global Economy Not Back to 2008',
 3: 'Finland GDP Expands In Q4',
 4: 'Tourism, govt spending buoys Thai economy in January',
 5: 'Intellitec Solutions to Host 13th Annual Spring Microsoft Dynamics User Group',
 6: 'Monday, 29 Feb 2016',
 7: 'Obama, stars pay a musical tribute to Ray Charles',
 8: 'Fire claims more than 100-year-old barn in Hancock County',
 9: "Microsoft's new Windows 10 ad targets Apple",
 10: 'Microsoft Project Centennial seen with “universal” desktop Office',
 11: 'Microsoft sneaks onto Android while Android sneaks onto Windows',
 12: 'Greek economy grows 0.1 pct q/q in fourth quarter',
 13: 'Big data and the Internet of Things to add £322BN to UK economy by 2020 – report',
 14: 'HoloLens dev edition costs £2,000 with new Conker game',
 15: 'Microsoft Word for Mac 2016 review: A subtle yet massive leap forward in usability and features',

In [53]:
words_list = []
for item in samples:
    sentence = list((str(samples[item])).lower())#将字母转化为小写
    for i in range(len(sentence)):
        if (not sentence[i].isalnum()) and sentence[i] != '\'':#用空格替换非字母
            sentence[i] = ' '
    sen = ''
    for i in sentence:
        sen += str(i)
    words_list += [list(sen.split())]
words_list

[['obama', 'lays', 'wreath', 'at', 'arlington', 'national', 'cemetery'],
 ['a', 'look', 'at', 'the', 'health', 'of', 'the', 'chinese', 'economy'],
 ['nouriel', 'roubini', 'global', 'economy', 'not', 'back', 'to', '2008'],
 ['finland', 'gdp', 'expands', 'in', 'q4'],
 ['tourism', 'govt', 'spending', 'buoys', 'thai', 'economy', 'in', 'january'],
 ['intellitec',
  'solutions',
  'to',
  'host',
  '13th',
  'annual',
  'spring',
  'microsoft',
  'dynamics',
  'user',
  'group'],
 ['monday', '29', 'feb', '2016'],
 ['obama', 'stars', 'pay', 'a', 'musical', 'tribute', 'to', 'ray', 'charles'],
 ['fire',
  'claims',
  'more',
  'than',
  '100',
  'year',
  'old',
  'barn',
  'in',
  'hancock',
  'county'],
 ["microsoft's", 'new', 'windows', '10', 'ad', 'targets', 'apple'],
 ['microsoft',
  'project',
  'centennial',
  'seen',
  'with',
  'universal',
  'desktop',
  'office'],
 ['microsoft',
  'sneaks',
  'onto',
  'android',
  'while',
  'android',
  'sneaks',
  'onto',
  'windows'],
 ['greek',


In [54]:
#读取停用词
with open('stopwords.txt','r') as f:
    stopwords = set(f.read().split())
print(stopwords)

{'here', "aren't", 'been', 'ought', 'herself', 'when', 'myself', 'after', 'on', 'l', 'they', 'm', 'than', "what's", 'must', 'its', 'or', 'ours', 'r', 'while', 'themselves', 'by', 'how', 'yourselves', "isn't", 'through', 'am', "weren't", 'have', 'very', 'z', 'shall', 'between', 'could', 'once', 'too', 'into', 'u', "here's", 'nor', 'during', 'why', 'd', "he's", "hasn't", 'more', "wasn't", 'us', "couldn't", 'she', 'about', 'other', "we'll", 'some', 't', 'doing', "she'll", 'in', 'would', 'will', 'i', 'our', 'off', 'n', 'this', 'so', 'him', 'yours', 'whom', 'against', 'your', 'a', 'onto', "didn't", "doesn't", 'are', 'their', 'j', 'being', 'them', 'who', "that's", 'o', 'further', 'was', 'with', 'you', 'at', 'both', 'f', 'h', 'be', 'has', 'no', 'does', 'any', 'q', 'where', "i'll", 'only', 'b', "mustn't", 'of', 'not', "we're", "there's", 'x', 'down', 'until', 'under', 'he', 'upon', "you're", 'having', "where's", 'each', 'himself', 'c', 'most', 'such', 'e', "i'd", 'out', 'then', "won't", "we've

In [55]:
#筛选非停用词
for  i in range(len(words_list)):
    new = []
    for j in words_list[i]:
        if j not in stopwords:
            new.append(j)
    words_list[i] = new
#处理一些异常字符
for i in range(len(words_list)):
    ls = words_list[i]
    for j in range(len(ls)):
        if ls[j][0] == '\'':
            if ls[j][-1] == '\'':
                ls[j] = ls[j][1:-1]
                continue
        if '\'' in ls[j]:
            ls[j] = ls[j][0:ls[j].index('\'')]
    words_list[i] = ls
words_list

[['obama', 'lays', 'wreath', 'arlington', 'national', 'cemetery'],
 ['look', 'health', 'chinese', 'economy'],
 ['nouriel', 'roubini', 'global', 'economy', 'back', '2008'],
 ['finland', 'gdp', 'expands', 'q4'],
 ['tourism', 'govt', 'spending', 'buoys', 'thai', 'economy', 'january'],
 ['intellitec',
  'solutions',
  'host',
  '13th',
  'annual',
  'spring',
  'microsoft',
  'dynamics',
  'user',
  'group'],
 ['monday', '29', 'feb', '2016'],
 ['obama', 'stars', 'pay', 'musical', 'tribute', 'ray', 'charles'],
 ['fire', 'claims', '100', 'year', 'old', 'barn', 'hancock', 'county'],
 ['microsoft', 'new', 'windows', '10', 'ad', 'targets', 'apple'],
 ['microsoft',
  'project',
  'centennial',
  'seen',
  'universal',
  'desktop',
  'office'],
 ['microsoft', 'sneaks', 'android', 'android', 'sneaks', 'windows'],
 ['greek', 'economy', 'grows', '0', '1', 'pct', 'fourth', 'quarter'],
 ['big',
  'data',
  'internet',
  'things',
  'add',
  '322bn',
  'uk',
  'economy',
  '2020',
  'report'],
 ['holol

In [56]:
#get_wordnet_pos函数是获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
#生成词形还原对象
wordnet_lemmatizer = WordNetLemmatizer()

In [57]:
#wordnet.synsets('roubini')#在词典里查找单词

In [58]:
#做词形还原处理
for item in words_list:
    for i in range(len(item)):
        try:
            item[i] = wordnet_lemmatizer.lemmatize(item[i],pos=get_wordnet_pos(pos_tag([item[i]])[0][1]))
        except:
            pass

In [59]:
words_list

[['obama', 'lay', 'wreath', 'arlington', 'national', 'cemetery'],
 ['look', 'health', 'chinese', 'economy'],
 ['nouriel', 'roubini', 'global', 'economy', 'back', '2008'],
 ['finland', 'gdp', 'expands', 'q4'],
 ['tourism', 'govt', 'spending', 'buoy', 'thai', 'economy', 'january'],
 ['intellitec',
  'solution',
  'host',
  '13th',
  'annual',
  'spring',
  'microsoft',
  'dynamic',
  'user',
  'group'],
 ['monday', '29', 'feb', '2016'],
 ['obama', 'star', 'pay', 'musical', 'tribute', 'ray', 'charles'],
 ['fire', 'claim', '100', 'year', 'old', 'barn', 'hancock', 'county'],
 ['microsoft', 'new', 'window', '10', 'ad', 'target', 'apple'],
 ['microsoft',
  'project',
  'centennial',
  'see',
  'universal',
  'desktop',
  'office'],
 ['microsoft', 'sneak', 'android', 'android', 'sneak', 'window'],
 ['greek', 'economy', 'grows', '0', '1', 'pct', 'fourth', 'quarter'],
 ['big',
  'data',
  'internet',
  'thing',
  'add',
  '322bn',
  'uk',
  'economy',
  '2020',
  'report'],
 ['hololens', 'dev', 

In [60]:
#处理某个异常词
for item in words_list:
    if "'slightly'" in item:
        print(item)
        item[item.index("'slightly'")] = 'slightly'
    if '' in item:
        print(item)
        item.remove('')

['buffett', 'politician', '', 'wrong', 'economy']
['microsoft', 'ad', 'target', 'apple', 'claim', 'window', '10', 'pc', '', 'more', 'mac']
['microsoft', 'ad', 'target', 'apple', 'claim', 'window', '10', 'pc', '', 'more', 'mac', 'aapl', 'msft']
['microsoft', 'ad', 'target', 'apple', 'claim', 'window', '10', 'pc', '', 'more', 'mac', 'aapl', 'msft']
['fed', 'dudley', 'risk', 'economy', 'tilt', "'slightly'", 'downside']
['market', 'orient', 'group', 'want', 'speed', '', 'lifetime', 'clean', 'economy', 'transition']
['market', 'orient', 'group', 'want', 'speed', '', 'lifetime', 'clean', 'economy', 'transition']
['market', 'orient', 'group', 'want', 'speed', '', 'lifetime', 'clean', 'economy', 'transition']
['microsoft', 'change', 'window', '10', '', 'upgrade', 'rule']
['microsoft', 'gutsy', 'challenge', 'apple', 'hardware', '', 'best', 'versus']
['microsoft', 'team', 'pc', 'maker', 'launch', '']
['microsoft', '', '200', 'old', 'laptop']
['', 'economy', 'strong', 'reputation']
['bank', 'eu',

In [61]:
for item in words_list:
    if '' in item:
        print(item)
        item.remove('')

['responsible', 'crime', 'commit', 'palestine', '', 'hezbollah']
['yellen', 'economy', 'well', '', 'december', 'live', 'rate', 'hike']
['obama', 'now', 'bibi', '']


In [62]:
for item in words_list:
    if '' in item:
        print(item)
        item.remove('')

In [63]:
#收集所有词
words=[]
for i in words_list:
    words+=i
words=sorted(list(set(words)))

In [64]:
words

['0',
 '000',
 '1',
 '10',
 '100',
 '1000',
 '100m',
 '104',
 '11',
 '12',
 '13',
 '13mn',
 '13th',
 '17',
 '175',
 '1868',
 '1937',
 '1948',
 '1q16',
 '2',
 '20',
 '200',
 '2008',
 '200gb',
 '2015',
 '2016',
 '2017',
 '2020',
 '2025',
 '20th',
 '271',
 '271000',
 '29',
 '3',
 '30',
 '300',
 '3000',
 '300b',
 '30th',
 '322bn',
 '360',
 '365',
 '3q',
 '4',
 '400',
 '4th',
 '5',
 '50',
 '6',
 '7',
 '70',
 '73m',
 '77',
 '7th',
 '8',
 '800',
 '80m',
 '900',
 '950',
 'aapl',
 'aaron',
 'abandon',
 'abbas',
 'abenomics',
 'abunimah',
 'academic',
 'accelerate',
 'accelerator',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accord',
 'acknowledge',
 'acquire',
 'acquires',
 'acquisition',
 'act',
 'action',
 'activist',
 'actually',
 'ad',
 'add',
 'address',
 'adjustment',
 'administration',
 'admits',
 'adore',
 'advance',
 'advertising',
 'adviser',
 'advocacy',
 'affair',
 'afsc',
 'agbaria',
 'agency',
 'agenda',
 'aggressively',
 'agile',
 'agree',
 'agreement',
 'ahead',
 'ai'

In [65]:
#生成单词索引
words_dict = {}
for i in range(len(words)):
    words_dict[words[i]] = i

In [66]:
print(words_dict)

{'0': 0, '000': 1, '1': 2, '10': 3, '100': 4, '1000': 5, '100m': 6, '104': 7, '11': 8, '12': 9, '13': 10, '13mn': 11, '13th': 12, '17': 13, '175': 14, '1868': 15, '1937': 16, '1948': 17, '1q16': 18, '2': 19, '20': 20, '200': 21, '2008': 22, '200gb': 23, '2015': 24, '2016': 25, '2017': 26, '2020': 27, '2025': 28, '20th': 29, '271': 30, '271000': 31, '29': 32, '3': 33, '30': 34, '300': 35, '3000': 36, '300b': 37, '30th': 38, '322bn': 39, '360': 40, '365': 41, '3q': 42, '4': 43, '400': 44, '4th': 45, '5': 46, '50': 47, '6': 48, '7': 49, '70': 50, '73m': 51, '77': 52, '7th': 53, '8': 54, '800': 55, '80m': 56, '900': 57, '950': 58, 'aapl': 59, 'aaron': 60, 'abandon': 61, 'abbas': 62, 'abenomics': 63, 'abunimah': 64, 'academic': 65, 'accelerate': 66, 'accelerator': 67, 'accepts': 68, 'access': 69, 'accident': 70, 'accidentally': 71, 'accord': 72, 'acknowledge': 73, 'acquire': 74, 'acquires': 75, 'acquisition': 76, 'act': 77, 'action': 78, 'activist': 79, 'actually': 80, 'ad': 81, 'add': 82, 

In [67]:
#格式为id:词语
ids_dict={}
for i in words_dict:
    ids_dict[words_dict[i]]=i

In [68]:
ids_dict

{0: '0',
 1: '000',
 2: '1',
 3: '10',
 4: '100',
 5: '1000',
 6: '100m',
 7: '104',
 8: '11',
 9: '12',
 10: '13',
 11: '13mn',
 12: '13th',
 13: '17',
 14: '175',
 15: '1868',
 16: '1937',
 17: '1948',
 18: '1q16',
 19: '2',
 20: '20',
 21: '200',
 22: '2008',
 23: '200gb',
 24: '2015',
 25: '2016',
 26: '2017',
 27: '2020',
 28: '2025',
 29: '20th',
 30: '271',
 31: '271000',
 32: '29',
 33: '3',
 34: '30',
 35: '300',
 36: '3000',
 37: '300b',
 38: '30th',
 39: '322bn',
 40: '360',
 41: '365',
 42: '3q',
 43: '4',
 44: '400',
 45: '4th',
 46: '5',
 47: '50',
 48: '6',
 49: '7',
 50: '70',
 51: '73m',
 52: '77',
 53: '7th',
 54: '8',
 55: '800',
 56: '80m',
 57: '900',
 58: '950',
 59: 'aapl',
 60: 'aaron',
 61: 'abandon',
 62: 'abbas',
 63: 'abenomics',
 64: 'abunimah',
 65: 'academic',
 66: 'accelerate',
 67: 'accelerator',
 68: 'accepts',
 69: 'access',
 70: 'accident',
 71: 'accidentally',
 72: 'accord',
 73: 'acknowledge',
 74: 'acquire',
 75: 'acquires',
 76: 'acquisition',
 7

In [69]:
find_words=list(map(set,words_list))#建立每篇文章的词语集，用集合查找加快速度

In [70]:
#统计每个单词的文档频率df
words_df = {}
for word in words_dict:
    num = 0
    for sentence in find_words:
        if word in sentence:
            num += 1
    words_df[words_dict[word]] = num

In [71]:
words_df

{0: 10,
 1: 13,
 2: 27,
 3: 40,
 4: 6,
 5: 1,
 6: 3,
 7: 6,
 8: 1,
 9: 2,
 10: 2,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 6,
 19: 26,
 20: 1,
 21: 2,
 22: 1,
 23: 1,
 24: 6,
 25: 14,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 2,
 33: 28,
 34: 7,
 35: 3,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 3,
 41: 6,
 42: 1,
 43: 22,
 44: 1,
 45: 1,
 46: 21,
 47: 1,
 48: 4,
 49: 8,
 50: 1,
 51: 1,
 52: 2,
 53: 2,
 54: 5,
 55: 1,
 56: 1,
 57: 1,
 58: 3,
 59: 2,
 60: 2,
 61: 1,
 62: 2,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 2,
 68: 1,
 69: 2,
 70: 1,
 71: 2,
 72: 1,
 73: 1,
 74: 2,
 75: 2,
 76: 2,
 77: 1,
 78: 2,
 79: 4,
 80: 2,
 81: 10,
 82: 8,
 83: 4,
 84: 1,
 85: 1,
 86: 1,
 87: 2,
 88: 1,
 89: 2,
 90: 1,
 91: 1,
 92: 3,
 93: 1,
 94: 1,
 95: 2,
 96: 2,
 97: 1,
 98: 1,
 99: 2,
 100: 1,
 101: 8,
 102: 1,
 103: 2,
 104: 2,
 105: 1,
 106: 1,
 107: 2,
 108: 1,
 109: 3,
 110: 1,
 111: 2,
 112: 1,
 113: 1,
 114: 1,
 115: 2,
 116: 4,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 1

In [72]:
#剔除df小于5的单词
select=[]
for word in words_df:
    if words_df[word]<=-1:
        select.append(word)

In [73]:
select

[]

In [74]:
for word in select:
    words_df.pop(word)

In [75]:
words_df

{0: 10,
 1: 13,
 2: 27,
 3: 40,
 4: 6,
 5: 1,
 6: 3,
 7: 6,
 8: 1,
 9: 2,
 10: 2,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 6,
 19: 26,
 20: 1,
 21: 2,
 22: 1,
 23: 1,
 24: 6,
 25: 14,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 2,
 33: 28,
 34: 7,
 35: 3,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 3,
 41: 6,
 42: 1,
 43: 22,
 44: 1,
 45: 1,
 46: 21,
 47: 1,
 48: 4,
 49: 8,
 50: 1,
 51: 1,
 52: 2,
 53: 2,
 54: 5,
 55: 1,
 56: 1,
 57: 1,
 58: 3,
 59: 2,
 60: 2,
 61: 1,
 62: 2,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 2,
 68: 1,
 69: 2,
 70: 1,
 71: 2,
 72: 1,
 73: 1,
 74: 2,
 75: 2,
 76: 2,
 77: 1,
 78: 2,
 79: 4,
 80: 2,
 81: 10,
 82: 8,
 83: 4,
 84: 1,
 85: 1,
 86: 1,
 87: 2,
 88: 1,
 89: 2,
 90: 1,
 91: 1,
 92: 3,
 93: 1,
 94: 1,
 95: 2,
 96: 2,
 97: 1,
 98: 1,
 99: 2,
 100: 1,
 101: 8,
 102: 1,
 103: 2,
 104: 2,
 105: 1,
 106: 1,
 107: 2,
 108: 1,
 109: 3,
 110: 1,
 111: 2,
 112: 1,
 113: 1,
 114: 1,
 115: 2,
 116: 4,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 1

In [76]:
words=[]
for i in words_df:
    words.append(ids_dict[i])

In [77]:
words

['0',
 '000',
 '1',
 '10',
 '100',
 '1000',
 '100m',
 '104',
 '11',
 '12',
 '13',
 '13mn',
 '13th',
 '17',
 '175',
 '1868',
 '1937',
 '1948',
 '1q16',
 '2',
 '20',
 '200',
 '2008',
 '200gb',
 '2015',
 '2016',
 '2017',
 '2020',
 '2025',
 '20th',
 '271',
 '271000',
 '29',
 '3',
 '30',
 '300',
 '3000',
 '300b',
 '30th',
 '322bn',
 '360',
 '365',
 '3q',
 '4',
 '400',
 '4th',
 '5',
 '50',
 '6',
 '7',
 '70',
 '73m',
 '77',
 '7th',
 '8',
 '800',
 '80m',
 '900',
 '950',
 'aapl',
 'aaron',
 'abandon',
 'abbas',
 'abenomics',
 'abunimah',
 'academic',
 'accelerate',
 'accelerator',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accord',
 'acknowledge',
 'acquire',
 'acquires',
 'acquisition',
 'act',
 'action',
 'activist',
 'actually',
 'ad',
 'add',
 'address',
 'adjustment',
 'administration',
 'admits',
 'adore',
 'advance',
 'advertising',
 'adviser',
 'advocacy',
 'affair',
 'afsc',
 'agbaria',
 'agency',
 'agenda',
 'aggressively',
 'agile',
 'agree',
 'agreement',
 'ahead',
 'ai'

In [78]:
#生成单词索引
Words_Dict = {}
for i in range(len(words)):
    Words_Dict[words[i]] = i

In [79]:
Words_Dict

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '1000': 5,
 '100m': 6,
 '104': 7,
 '11': 8,
 '12': 9,
 '13': 10,
 '13mn': 11,
 '13th': 12,
 '17': 13,
 '175': 14,
 '1868': 15,
 '1937': 16,
 '1948': 17,
 '1q16': 18,
 '2': 19,
 '20': 20,
 '200': 21,
 '2008': 22,
 '200gb': 23,
 '2015': 24,
 '2016': 25,
 '2017': 26,
 '2020': 27,
 '2025': 28,
 '20th': 29,
 '271': 30,
 '271000': 31,
 '29': 32,
 '3': 33,
 '30': 34,
 '300': 35,
 '3000': 36,
 '300b': 37,
 '30th': 38,
 '322bn': 39,
 '360': 40,
 '365': 41,
 '3q': 42,
 '4': 43,
 '400': 44,
 '4th': 45,
 '5': 46,
 '50': 47,
 '6': 48,
 '7': 49,
 '70': 50,
 '73m': 51,
 '77': 52,
 '7th': 53,
 '8': 54,
 '800': 55,
 '80m': 56,
 '900': 57,
 '950': 58,
 'aapl': 59,
 'aaron': 60,
 'abandon': 61,
 'abbas': 62,
 'abenomics': 63,
 'abunimah': 64,
 'academic': 65,
 'accelerate': 66,
 'accelerator': 67,
 'accepts': 68,
 'access': 69,
 'accident': 70,
 'accidentally': 71,
 'accord': 72,
 'acknowledge': 73,
 'acquire': 74,
 'acquires': 75,
 'acquisition': 76,
 '

In [80]:
#统计每个单词的文档频率df
Words_DF = {}
for word in words_df:
    Words_DF[Words_Dict[ids_dict[word]]]=words_df[word]

In [81]:
Words_DF

{0: 10,
 1: 13,
 2: 27,
 3: 40,
 4: 6,
 5: 1,
 6: 3,
 7: 6,
 8: 1,
 9: 2,
 10: 2,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 6,
 19: 26,
 20: 1,
 21: 2,
 22: 1,
 23: 1,
 24: 6,
 25: 14,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 2,
 33: 28,
 34: 7,
 35: 3,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 3,
 41: 6,
 42: 1,
 43: 22,
 44: 1,
 45: 1,
 46: 21,
 47: 1,
 48: 4,
 49: 8,
 50: 1,
 51: 1,
 52: 2,
 53: 2,
 54: 5,
 55: 1,
 56: 1,
 57: 1,
 58: 3,
 59: 2,
 60: 2,
 61: 1,
 62: 2,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 2,
 68: 1,
 69: 2,
 70: 1,
 71: 2,
 72: 1,
 73: 1,
 74: 2,
 75: 2,
 76: 2,
 77: 1,
 78: 2,
 79: 4,
 80: 2,
 81: 10,
 82: 8,
 83: 4,
 84: 1,
 85: 1,
 86: 1,
 87: 2,
 88: 1,
 89: 2,
 90: 1,
 91: 1,
 92: 3,
 93: 1,
 94: 1,
 95: 2,
 96: 2,
 97: 1,
 98: 1,
 99: 2,
 100: 1,
 101: 8,
 102: 1,
 103: 2,
 104: 2,
 105: 1,
 106: 1,
 107: 2,
 108: 1,
 109: 3,
 110: 1,
 111: 2,
 112: 1,
 113: 1,
 114: 1,
 115: 2,
 116: 4,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 1

In [82]:
#格式为ID:词语
IDs_Dict={}
for i in Words_Dict:
    IDs_Dict[Words_Dict[i]]=i

In [83]:
IDs_Dict

{0: '0',
 1: '000',
 2: '1',
 3: '10',
 4: '100',
 5: '1000',
 6: '100m',
 7: '104',
 8: '11',
 9: '12',
 10: '13',
 11: '13mn',
 12: '13th',
 13: '17',
 14: '175',
 15: '1868',
 16: '1937',
 17: '1948',
 18: '1q16',
 19: '2',
 20: '20',
 21: '200',
 22: '2008',
 23: '200gb',
 24: '2015',
 25: '2016',
 26: '2017',
 27: '2020',
 28: '2025',
 29: '20th',
 30: '271',
 31: '271000',
 32: '29',
 33: '3',
 34: '30',
 35: '300',
 36: '3000',
 37: '300b',
 38: '30th',
 39: '322bn',
 40: '360',
 41: '365',
 42: '3q',
 43: '4',
 44: '400',
 45: '4th',
 46: '5',
 47: '50',
 48: '6',
 49: '7',
 50: '70',
 51: '73m',
 52: '77',
 53: '7th',
 54: '8',
 55: '800',
 56: '80m',
 57: '900',
 58: '950',
 59: 'aapl',
 60: 'aaron',
 61: 'abandon',
 62: 'abbas',
 63: 'abenomics',
 64: 'abunimah',
 65: 'academic',
 66: 'accelerate',
 67: 'accelerator',
 68: 'accepts',
 69: 'access',
 70: 'accident',
 71: 'accidentally',
 72: 'accord',
 73: 'acknowledge',
 74: 'acquire',
 75: 'acquires',
 76: 'acquisition',
 7

In [84]:
words_set=set(words)

In [85]:
words_set

{'surrender',
 'hail',
 'weighs',
 'plo',
 'satnad',
 'magic',
 'resolve',
 'secret',
 'crude',
 'bendy',
 'internet',
 'drama',
 'job',
 'keystone',
 'tinker',
 'save',
 '20th',
 'flight',
 'seattle',
 'decline',
 'short',
 'coca',
 'among',
 'video',
 'madagascar',
 'reclaims',
 'pace',
 'election',
 'gc',
 'wc18',
 'maersk',
 'ready',
 'dx',
 'club',
 'management',
 'expect',
 'leave',
 'indonesia',
 '1000',
 'launcher',
 'currency',
 'central',
 'rhetoric',
 'coverage',
 'allege',
 'authorize',
 'williams',
 'gig',
 'cowboy',
 'effectively',
 'ray',
 'confederation',
 'hump',
 'progress',
 'growth',
 'dead',
 'slow',
 'add',
 'pattern',
 'prison',
 'weak',
 'middle',
 'pan',
 'toronto',
 'viewpoint',
 'education',
 'false',
 'unrwa',
 'yet',
 'emergence',
 'play',
 'redmond',
 'blur',
 'amp',
 'lawyer',
 'file',
 'date',
 'heartlessness',
 'pak',
 'zacks',
 'wound',
 'commitment',
 'kerry',
 '2008',
 'hurdle',
 'language',
 'briefly',
 'text',
 'fifth',
 'ramzy',
 'ship',
 'organ',

In [86]:
Words_List=[]
for i in range(len(words_list)):
    new=[]
    for j in words_list[i]:
        if j in  words_set:
            new.append(j)
    Words_List.append(new)

In [87]:
Words_List

[['obama', 'lay', 'wreath', 'arlington', 'national', 'cemetery'],
 ['look', 'health', 'chinese', 'economy'],
 ['nouriel', 'roubini', 'global', 'economy', 'back', '2008'],
 ['finland', 'gdp', 'expands', 'q4'],
 ['tourism', 'govt', 'spending', 'buoy', 'thai', 'economy', 'january'],
 ['intellitec',
  'solution',
  'host',
  '13th',
  'annual',
  'spring',
  'microsoft',
  'dynamic',
  'user',
  'group'],
 ['monday', '29', 'feb', '2016'],
 ['obama', 'star', 'pay', 'musical', 'tribute', 'ray', 'charles'],
 ['fire', 'claim', '100', 'year', 'old', 'barn', 'hancock', 'county'],
 ['microsoft', 'new', 'window', '10', 'ad', 'target', 'apple'],
 ['microsoft',
  'project',
  'centennial',
  'see',
  'universal',
  'desktop',
  'office'],
 ['microsoft', 'sneak', 'android', 'android', 'sneak', 'window'],
 ['greek', 'economy', 'grows', '0', '1', 'pct', 'fourth', 'quarter'],
 ['big',
  'data',
  'internet',
  'thing',
  'add',
  '322bn',
  'uk',
  'economy',
  '2020',
  'report'],
 ['hololens', 'dev', 

In [88]:
#将单词替换为代号
words_copy = Words_List
for sentence in words_copy:
    for i in range(len(sentence)):
        sentence[i] = Words_Dict[sentence[i]]

In [89]:
words_copy

[[1291, 1085, 2069, 162, 1255, 353],
 [1133, 888, 380, 632],
 [1280, 1605, 825, 632, 198, 22],
 [744, 815, 687, 1473],
 [1909, 841, 1759, 310, 1880, 632, 1023],
 [984, 1740, 921, 12, 144, 1763, 1197, 619, 1980, 851],
 [1227, 32, 724, 25],
 [1291, 1776, 1368, 1246, 1923, 1499, 369],
 [746, 393, 4, 2076, 1306, 211, 868, 476],
 [1197, 1268, 2050, 3, 81, 1855, 154],
 [1197, 1452, 355, 1653, 1956, 560, 1302],
 [1197, 1731, 138, 138, 1731, 2050],
 [848, 632, 853, 0, 2, 1371, 790, 1478],
 [239, 516, 987, 1884, 82, 39, 1948, 632, 27, 1550],
 [913, 565, 636, 472, 19, 1, 1268, 450, 808],
 [1197, 2059, 1141, 25, 1582, 1807, 2078, 1172, 1091, 785, 1977, 723],
 [1197, 207, 19, 806, 153, 1138, 1434],
 [1197, 1429, 1958, 1302, 25, 800, 560, 156, 2050, 1795],
 [848, 632, 1691, 0, 33, 1378, 1078, 2076, 686, 1910, 653],
 [1836, 632, 746, 1874, 1270, 475, 357, 209],
 [1197, 207, 19, 1283, 472, 1045, 14],
 [848, 632, 1691, 0, 33, 1378, 1078, 2076, 686, 1910, 653],
 [379, 508, 1559, 1556, 238, 267, 632],
 

In [90]:
#计算每篇文章的词频
from collections import Counter
Words_TF = {}
for i in range(len(words_copy)):
    Words_TF[i] = dict(Counter(words_copy[i]))

In [91]:
Words_TF

{0: {162: 1, 353: 1, 1085: 1, 1255: 1, 1291: 1, 2069: 1},
 1: {380: 1, 632: 1, 888: 1, 1133: 1},
 2: {22: 1, 198: 1, 632: 1, 825: 1, 1280: 1, 1605: 1},
 3: {687: 1, 744: 1, 815: 1, 1473: 1},
 4: {310: 1, 632: 1, 841: 1, 1023: 1, 1759: 1, 1880: 1, 1909: 1},
 5: {12: 1,
  144: 1,
  619: 1,
  851: 1,
  921: 1,
  984: 1,
  1197: 1,
  1740: 1,
  1763: 1,
  1980: 1},
 6: {25: 1, 32: 1, 724: 1, 1227: 1},
 7: {369: 1, 1246: 1, 1291: 1, 1368: 1, 1499: 1, 1776: 1, 1923: 1},
 8: {4: 1, 211: 1, 393: 1, 476: 1, 746: 1, 868: 1, 1306: 1, 2076: 1},
 9: {3: 1, 81: 1, 154: 1, 1197: 1, 1268: 1, 1855: 1, 2050: 1},
 10: {355: 1, 560: 1, 1197: 1, 1302: 1, 1452: 1, 1653: 1, 1956: 1},
 11: {138: 2, 1197: 1, 1731: 2, 2050: 1},
 12: {0: 1, 2: 1, 632: 1, 790: 1, 848: 1, 853: 1, 1371: 1, 1478: 1},
 13: {27: 1,
  39: 1,
  82: 1,
  239: 1,
  516: 1,
  632: 1,
  987: 1,
  1550: 1,
  1884: 1,
  1948: 1},
 14: {1: 1, 19: 1, 450: 1, 472: 1, 565: 1, 636: 1, 808: 1, 913: 1, 1268: 1},
 15: {25: 1,
  723: 1,
  785: 1,
  10

In [92]:
#计算得到各个文章的每个单词的特征权重tf/df
Words_TF_DF = Words_TF
for sentence in Words_TF_DF.values():
    for word in sentence:
        sentence[word] = sentence[word] / Words_DF[word]

In [93]:
Words_TF_DF

{0: {162: 1.0,
  353: 0.5,
  1085: 0.5,
  1255: 0.5,
  1291: 0.038461538461538464,
  2069: 1.0},
 1: {380: 0.25, 632: 0.003787878787878788, 888: 0.5, 1133: 0.2},
 2: {22: 1.0,
  198: 0.08333333333333333,
  632: 0.003787878787878788,
  825: 0.047619047619047616,
  1280: 1.0,
  1605: 1.0},
 3: {687: 0.3333333333333333, 744: 0.5, 815: 0.3333333333333333, 1473: 0.5},
 4: {310: 1.0,
  632: 0.003787878787878788,
  841: 0.5,
  1023: 0.5,
  1759: 0.5,
  1880: 1.0,
  1909: 1.0},
 5: {12: 1.0,
  144: 0.5,
  619: 0.5,
  851: 0.16666666666666666,
  921: 0.09090909090909091,
  984: 1.0,
  1197: 0.0023923444976076554,
  1740: 0.25,
  1763: 1.0,
  1980: 0.1111111111111111},
 6: {25: 0.07142857142857142, 32: 0.5, 724: 0.5, 1227: 0.3333333333333333},
 7: {369: 1.0,
  1246: 1.0,
  1291: 0.038461538461538464,
  1368: 0.2,
  1499: 1.0,
  1776: 0.5,
  1923: 1.0},
 8: {4: 0.16666666666666666,
  211: 1.0,
  393: 0.25,
  476: 0.5,
  746: 0.25,
  868: 1.0,
  1306: 0.14285714285714285,
  2076: 0.066666666666666

In [94]:
#用于查询
#list(Words_Dict.keys())[list (Words_Dict.values()).index (653)]

In [95]:
#存入json
import json
with open('Words_Dict.json','w') as f:
        f.write(json.dumps(Words_Dict))
with open('IDs_Dict.json','w') as f:
        f.write(json.dumps(IDs_Dict))
with open('Words_TF_DF.json','w') as f:
        f.write(json.dumps(Words_TF_DF))

In [96]:
#查看字典
with open('Words_Dict.json','r') as f:
    json1=json.load(f)
with open('IDs_Dict.json','r') as f:
    json2=json.load(f)
with open('Words_TF_DF.json','r') as f:
    json3=json.load(f)

In [97]:
json1

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '1000': 5,
 '100m': 6,
 '104': 7,
 '11': 8,
 '12': 9,
 '13': 10,
 '13mn': 11,
 '13th': 12,
 '17': 13,
 '175': 14,
 '1868': 15,
 '1937': 16,
 '1948': 17,
 '1q16': 18,
 '2': 19,
 '20': 20,
 '200': 21,
 '2008': 22,
 '200gb': 23,
 '2015': 24,
 '2016': 25,
 '2017': 26,
 '2020': 27,
 '2025': 28,
 '20th': 29,
 '271': 30,
 '271000': 31,
 '29': 32,
 '3': 33,
 '30': 34,
 '300': 35,
 '3000': 36,
 '300b': 37,
 '30th': 38,
 '322bn': 39,
 '360': 40,
 '365': 41,
 '3q': 42,
 '4': 43,
 '400': 44,
 '4th': 45,
 '5': 46,
 '50': 47,
 '6': 48,
 '7': 49,
 '70': 50,
 '73m': 51,
 '77': 52,
 '7th': 53,
 '8': 54,
 '800': 55,
 '80m': 56,
 '900': 57,
 '950': 58,
 'aapl': 59,
 'aaron': 60,
 'abandon': 61,
 'abbas': 62,
 'abenomics': 63,
 'abunimah': 64,
 'academic': 65,
 'accelerate': 66,
 'accelerator': 67,
 'accepts': 68,
 'access': 69,
 'accident': 70,
 'accidentally': 71,
 'accord': 72,
 'acknowledge': 73,
 'acquire': 74,
 'acquires': 75,
 'acquisition': 76,
 '

In [98]:
json2

{'0': '0',
 '1': '000',
 '2': '1',
 '3': '10',
 '4': '100',
 '5': '1000',
 '6': '100m',
 '7': '104',
 '8': '11',
 '9': '12',
 '10': '13',
 '11': '13mn',
 '12': '13th',
 '13': '17',
 '14': '175',
 '15': '1868',
 '16': '1937',
 '17': '1948',
 '18': '1q16',
 '19': '2',
 '20': '20',
 '21': '200',
 '22': '2008',
 '23': '200gb',
 '24': '2015',
 '25': '2016',
 '26': '2017',
 '27': '2020',
 '28': '2025',
 '29': '20th',
 '30': '271',
 '31': '271000',
 '32': '29',
 '33': '3',
 '34': '30',
 '35': '300',
 '36': '3000',
 '37': '300b',
 '38': '30th',
 '39': '322bn',
 '40': '360',
 '41': '365',
 '42': '3q',
 '43': '4',
 '44': '400',
 '45': '4th',
 '46': '5',
 '47': '50',
 '48': '6',
 '49': '7',
 '50': '70',
 '51': '73m',
 '52': '77',
 '53': '7th',
 '54': '8',
 '55': '800',
 '56': '80m',
 '57': '900',
 '58': '950',
 '59': 'aapl',
 '60': 'aaron',
 '61': 'abandon',
 '62': 'abbas',
 '63': 'abenomics',
 '64': 'abunimah',
 '65': 'academic',
 '66': 'accelerate',
 '67': 'accelerator',
 '68': 'accepts',
 '69'

In [99]:
json3

{'0': {'1085': 0.5,
  '1255': 0.5,
  '1291': 0.038461538461538464,
  '162': 1.0,
  '2069': 1.0,
  '353': 0.5},
 '1': {'1133': 0.2, '380': 0.25, '632': 0.003787878787878788, '888': 0.5},
 '2': {'1280': 1.0,
  '1605': 1.0,
  '198': 0.08333333333333333,
  '22': 1.0,
  '632': 0.003787878787878788,
  '825': 0.047619047619047616},
 '3': {'1473': 0.5,
  '687': 0.3333333333333333,
  '744': 0.5,
  '815': 0.3333333333333333},
 '4': {'1023': 0.5,
  '1759': 0.5,
  '1880': 1.0,
  '1909': 1.0,
  '310': 1.0,
  '632': 0.003787878787878788,
  '841': 0.5},
 '5': {'1197': 0.0023923444976076554,
  '12': 1.0,
  '144': 0.5,
  '1740': 0.25,
  '1763': 1.0,
  '1980': 0.1111111111111111,
  '619': 0.5,
  '851': 0.16666666666666666,
  '921': 0.09090909090909091,
  '984': 1.0},
 '6': {'1227': 0.3333333333333333,
  '25': 0.07142857142857142,
  '32': 0.5,
  '724': 0.5},
 '7': {'1246': 1.0,
  '1291': 0.038461538461538464,
  '1368': 0.2,
  '1499': 1.0,
  '1776': 0.5,
  '1923': 1.0,
  '369': 1.0},
 '8': {'1306': 0.1428

In [100]:
len(json1)

2087